In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio
import torchaudio.transforms as T
import torchaudio.functional as F_audio
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from read_data import RIRData
from sklearn.model_selection import train_test_split

In [4]:
data = RIRData("/home/lucas/ML/gtu-rir/RIR.pickle.dat")

all_rir_values = [entry[data.rir_data_field_numbers["rirData"]] for entry in data.rir_data]


rirData Length =15202


In [ ]:
# Get a list of all the MFCC for all the RIRs
all_mfcc = []
for i in range(len(all_rir_values)):
    # Get the MFCC for the current RIR
    mfcc = T.MFCC(
        sample_rate=22050,
        n_mfcc=20,
        melkwargs={"n_fft": 2048, "hop_length": 512, "n_mels": 128},
    )
    mfcc_values = mfcc(torch.tensor(all_rir_values[i], dtype=torch.float32))
    all_mfcc.append(mfcc_values)

# print the shape of the MFCC values
print("MFCC shape:", all_mfcc[0].shape) # (20, 173)

MFCC shape: torch.Size([20, 173])


In [ ]:
# normalize the MFCC values
all_mfcc = [F.normalize(mfcc, p=2, dim=1) for mfcc in all_mfcc]
# print the shape of the normalized MFCC values
print("Normalized MFCC shape:", all_mfcc[0].shape) # shape(20, 173)

Normalized MFCC shape: torch.Size([20, 173])


In [11]:
# Split the data into training and testing sets
train_mfcc, test_mfcc = train_test_split(all_mfcc, test_size=0.2, random_state=42)
# print the shapes of the training and testing sets
print("Training MFCC shape:", train_mfcc[0].shape)
print("Testing MFCC shape:", test_mfcc[0].shape)
# Create a dataset class
class RIRDataset(Dataset):
    def __init__(self, mfcc_data):
        self.mfcc_data = mfcc_data

    def __len__(self):
        return len(self.mfcc_data)

    def __getitem__(self, idx):
        return self.mfcc_data[idx]
# Create the training and testing datasets
train_dataset = RIRDataset(train_mfcc)
test_dataset = RIRDataset(test_mfcc)
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Training MFCC shape: torch.Size([20, 173])
Testing MFCC shape: torch.Size([20, 173])
